This is [Langchain](https://python.langchain.com/) toolkits allows agents to interact with APITable Fusion API, performing actions such as get spaces, get datasheets, get records, create fields and create datasheet. The tool wraps the [apitable.py](https://github.com/apitable/apitable-sdks/tree/develop/apitable.py) library. 

And it is in beta, please don't use it in production.

To use this tool, you must first set as environment variables: APITABLE_API_TOKEN
```python
os.environ["OPENAI_API_KEY"] = "xxx"
os.environ["APITABLE_API_TOKEN"] = "xxx"
```



In [18]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent

from langchain.chat_models import ChatOpenAI

from langchain.callbacks import get_openai_callback

from apitable_toolkit.tool.prompt import PREFIX, FORMAT_INSTRUCTIONS, SUFFIX
from apitable_toolkit.toolkit import APITableToolkit
from apitable_toolkit.utilities.apitable import APITableAPIWrapper


In [19]:
llm = ChatOpenAI(temperature=0)
apitable = APITableAPIWrapper()
toolkit = APITableToolkit.from_apitable_api_wrapper(apitable)
agent = initialize_agent(
    toolkit.get_tools(),
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={
        "prefix": PREFIX,
        "format_instructions": FORMAT_INSTRUCTIONS,
        "suffix": SUFFIX,
    },
)

In [20]:
def run(prompt):
    with get_openai_callback() as cb:
        try:
            agent.run(prompt)
        except Exception as e:
            print(e)
        print(f"Total Tokens: {cb.total_tokens}")
        print(f"Prompt Tokens: {cb.prompt_tokens}")
        print(f"Completion Tokens: {cb.completion_tokens}")
        print(f"Successful Requests: {cb.successful_requests}")
        print(f"Total Cost (USD): ${cb.total_cost}")

In [21]:
run("What spaces do I have")



> Entering new AgentExecutor chain...


KeyboardInterrupt: 

In [ ]:
run("Tell me the latest value of APITable MAU in xukecheng's space")



> Entering new AgentExecutor chain...
We need to find the latest value of APITable MAU in xukecheng's space. We can start by getting all the spaces xukecheng has access to and then searching for the specific datasheet in each space.
Action: Get Spaces
Action Input: None
Observation: Found 4 spaces:
[{'id': 'spcS0eZxZ8mSA', 'name': 'APITable Ltd.', 'isAdmin': None}, {'id': 'spctqtTZpssYw', 'name': "xukecheng's Space", 'isAdmin': True}, {'id': 'spcGSVizcwRYF', 'name': 'Gmail Space', 'isAdmin': None}, {'id': 'spcV5VkzU71Lf', 'name': 'test03-bug验证升级', 'isAdmin': None}]
Thought:We have found the space we need, now we need to search for the specific datasheet in xukecheng's space.
Action: Get Nodes
Action Input: "spctqtTZpssYw" (xukecheng's space id)
Observation: Found a error 'Expecting value: line 1 column 1 (char 0)', you should only output Action Input in JSON format
Thought:We need to input the space id in JSON format.
Action: Get Nodes
Action Input: {"space_id": "spctqtTZpssYw"}
Obse

In [30]:
run("Create a datasheet for Project management in xukecheng's space")



> Entering new AgentExecutor chain...
I need to create a datasheet in a specific space with a specific name and fields. I should use the Create Datasheets tool.
Action: Create Datasheets
Action Input: space_id = xukecheng's space id, name = "Project management"
Observation: Found a error 'Expecting value: line 1 column 1 (char 0)', you should only output Action Input in JSON format
Thought:I need to input the space_id and name in JSON format.
Action: Create Datasheets
Action Input: {"space_id": "xukecheng's space id", "name": "Project management"}
Observation: Found a error 'Action input need effective space_id', please try another tool to get right space_id.
Thought:I need to use the Get Spaces tool to retrieve the correct space_id.
Action: Get Spaces
Action Input: None
Observation: Found 4 spaces:
[{'id': 'spcS0eZxZ8mSA', 'name': 'APITable Ltd.', 'isAdmin': None}, {'id': 'spctqtTZpssYw', 'name': "xukecheng's Space", 'isAdmin': True}, {'id': 'spcGSVizcwRYF', 'name': 'Gmail Space', '

In [38]:
run("Create a datasheet for Project management in xukecheng's space, need to include the following fields: name, description, start_date, end_date, status, owner, priority, progress, comments")



> Entering new AgentExecutor chain...
I need to create a datasheet in xukecheng's space with the specified fields. I should use the Create Datasheets tool for this task.
Action: Create Datasheets
Action Input: 
- space id: xukecheng's space id
- name: "Project management"
- field_data: 
  - {"name": "name", "type": "SingleText", "property": {"defaultValue": ""}}
  - {"name": "description", "type": "Text"}
  - {"name": "start_date", "type": "DateTime", "property": {"dateFormat": "YYYY-MM-DD", "includeTime": false}}
  - {"name": "end_date", "type": "DateTime", "property": {"dateFormat": "YYYY-MM-DD", "includeTime": false}}
  - {"name": "status", "type": "SingleSelect", "property": {"options": [{"name": "Not started"}, {"name": "In progress"}, {"name": "Completed"}]}}
  - {"name": "owner", "type": "Member"}
  - {"name": "priority", "type": "SingleSelect", "property": {"options": [{"name": "Low"}, {"name": "Medium"}, {"name": "High"}]}}
  - {"name": "progress", "type": "Percent"}
  - {"n